## Setup 

In [22]:
import importlib
from pathlib import Path 
import sys 
import polars as pl 

PB_CPG_TOOL_MODE = 'model' # mode of aligned_bam_to_cpg_scores
OUTPUT_DIR = f"/scratch/ucgd/lustre-labs/quinlan/data-shared/dna-methylation/CEPH1463.GRCh38.hifi.{PB_CPG_TOOL_MODE}.founder-phased.all-cpgs" 
BED_ALL_CPGS = f"{OUTPUT_DIR}/all_cpg_sites.bed" # output of src/write_all_cpgs.py
METH_FOUNDER_PHASED_DIR = f"/scratch/ucgd/lustre-labs/quinlan/data-shared/dna-methylation/CEPH1463.GRCh38.hifi.{PB_CPG_TOOL_MODE}.founder-phased" # output dir of phase_meth_to_founder_haps.py
UID = 200081
BED_HAP_MAP = f"{METH_FOUNDER_PHASED_DIR}/{UID}.hap-map-blocks.bed" # bed file of hap-map blocks from src/phase_meth_to_founder_haps.py

REPO_DIR = Path('/scratch/ucgd/lustre-labs/quinlan/u6018199/tapestry')
sys.path.append(f"{REPO_DIR}/src/util") 

## Get all CpG sites

In [25]:
import expand_to_all_cpgs
importlib.reload(expand_to_all_cpgs)
from expand_to_all_cpgs import read_all_cpgs

DF_ALL_CPGS = read_all_cpgs(BED_ALL_CPGS)
DF_ALL_CPGS

chrom,start,end
str,i64,i64
"""chr1""",10470,10471
"""chr1""",10483,10484
"""chr1""",10488,10489
"""chr1""",10492,10493
"""chr1""",10496,10497
…,…,…
"""chr1""",921607,921608
"""chr1""",921617,921618
"""chr1""",921627,921628


## Get hap-map blocks

In [31]:
importlib.reload(expand_to_all_cpgs)
from expand_to_all_cpgs import read_hap_map_blocks

DF_HAP_MAP_BLOCKS = read_hap_map_blocks(BED_HAP_MAP)
DF_HAP_MAP_BLOCKS

chrom,start,end
str,i64,i64
"""chr1""",492339,532812
"""chr1""",593123,1314109
"""chr1""",1351126,1382658
"""chr1""",1421668,1427528
"""chr1""",1432961,1610923
…,…,…
"""chrX""",154941725,154993498
"""chrX""",155280990,155292580
"""chrX""",155370893,155485214


## Assign CpG sites to an overlapping hap-map block, if such exists 

In [ ]:
importlib.reload(expand_to_all_cpgs)
from expand_to_all_cpgs import assign_hap_map_blocks_to_cpgs

DF_ALL_CPGS_WITH_HAP_MAP_BLOCKS = assign_hap_map_blocks_to_cpgs(DF_ALL_CPGS, DF_HAP_MAP_BLOCKS)
DF_ALL_CPGS_WITH_HAP_MAP_BLOCKS 

chrom,start,end,start_hap_map_block,end_hap_map_block
str,i64,i64,i64,i64
"""chr1""",10470,10471,null,null
"""chr1""",10483,10484,null,null
"""chr1""",10488,10489,null,null
"""chr1""",10492,10493,null,null
"""chr1""",10496,10497,null,null
…,…,…,…,…
"""chr1""",921607,921608,593123,1314109
"""chr1""",921617,921618,593123,1314109
"""chr1""",921627,921628,593123,1314109


In [ ]:
# TODO: see TODOs in src/expand_to_all_cpgs.py

In [ ]:
# TODO: get expand_to_all_cpgs.sh working 